In [1]:
#llm model=qwen
#embedding model=all-MiniLM-L6-v2

!pip install langchain langchain-huggingface langchain_community langgraph langchain-text-splitters sentence-transformers

from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

from langchain_community.embeddings import HuggingFaceEmbeddings


from langchain_core.vectorstores import InMemoryVectorStore



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core

In [26]:


# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/#task-decomposition",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs=loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# ✅ Embedding model wrapped for LangChain
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ Vector store
vector_store = InMemoryVectorStore(embedding=embedding_model)
vector_store.add_documents(all_splits)

#llm model
llm=HuggingFacePipeline.from_model_id(
    #model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_id="Qwen/Qwen2.5-1.5B-Instruct",
    device_map="auto",
    task="text-generation",
)

model=ChatHuggingFace(llm=llm)

prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model.invoke(messages)
    return {"answer": response.content}


# Compile application
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve","generate")
graph = graph_builder.compile()

#test
response = graph.invoke({"question": "Why is task decomposition important for autonomous agents?"})
print(response["answer"])

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Why is task decomposition important for autonomous agents? 
Context: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, 

In [4]:
import csv

# List of Q&A dictionaries
qna_pairs = [
    {"question": "What is task decomposition in autonomous agents?",
     "answer": "Task decomposition involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks."},
    {"question": "Why is task decomposition important for autonomous agents?",
     "answer": "It allows agents to handle complex tasks by dividing them into simpler steps, facilitating better planning and execution."},
    {"question": "What is Chain of Thought (CoT) prompting?",
     "answer": "CoT is a prompting technique where models are instructed to 'think step by step,' enhancing performance on complex tasks by decomposing them into smaller steps."},
    {"question": "How does Tree of Thoughts (ToT) differ from CoT?",
     "answer": "ToT extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure with multiple thoughts per step."},
    {"question": "What are the search strategies used in ToT?",
     "answer": "ToT employs breadth-first search (BFS) or depth-first search (DFS) to evaluate different reasoning paths."},
    {"question": "How can LLMs perform task decomposition?",
     "answer": "LLMs can decompose tasks using simple prompts like 'Steps for XYZ' or by using task-specific instructions such as 'Write a story outline.'"},
    {"question": "What is the LLM+P approach?",
     "answer": "LLM+P involves using an external classical planner with the Planning Domain Definition Language (PDDL) to perform long-horizon planning, translating problems into PDDL and generating plans."},
    {"question": "What role does PDDL play in LLM+P?",
     "answer": "PDDL serves as an intermediate interface to describe planning problems, enabling LLMs to generate plans through external classical planners."},
    {"question": "Can LLMs perform task decomposition without external tools?",
     "answer": "Yes, LLMs can perform task decomposition using simple prompting techniques or task-specific instructions without relying on external tools."},
    {"question": "What is the significance of task decomposition in autonomous agents?",
     "answer": "It enables agents to break down complex tasks into manageable steps, improving efficiency and effectiveness in task execution."}
]

# Write to CSV
csv_file = "task_decomposition_qna.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["question", "answer"])
    writer.writeheader()
    for qna in qna_pairs:
        writer.writerow(qna)

print(f"CSV file '{csv_file}' created successfully!")


CSV file 'task_decomposition_qna.csv' created successfully!


In [5]:
#load your qna csv
import pandas as pd

qna_df = pd.read_csv("task_decomposition_qna.csv")
print(qna_df.head())


                                            question  \
0   What is task decomposition in autonomous agents?   
1  Why is task decomposition important for autono...   
2          What is Chain of Thought (CoT) prompting?   
3   How does Tree of Thoughts (ToT) differ from CoT?   
4        What are the search strategies used in ToT?   

                                              answer  
0  Task decomposition involves breaking down larg...  
1  It allows agents to handle complex tasks by di...  
2  CoT is a prompting technique where models are ...  
3  ToT extends CoT by exploring multiple reasonin...  
4  ToT employs breadth-first search (BFS) or dept...  


# Task
Benchmark the RAG model using the uploaded Q&A file.

## Define evaluation function

### Subtask:
Create a function that takes a question as input, invokes the RAG model to get an answer, and compares it to the expected answer from the Q&A dataset.


**Reasoning**:
I need to define a Python function that takes a question, invokes the RAG model, and returns the generated answer.



In [7]:
def get_rag_answer(question: str) -> str:
    """
    Invokes the RAG model with a given question and returns the generated answer.

    Args:
        question: The question to pass to the RAG model.

    Returns:
        The answer generated by the RAG model.
    """
    response = graph.invoke({"question": question})
    return response["answer"]

## Evaluate rag model

### Subtask:
Iterate through the questions in the Q&A dataset, apply the evaluation function to each question, and store the results.


**Reasoning**:
Initialize an empty list to store evaluation results and iterate through the qna_df to evaluate each question using the previously defined `get_rag_answer` function.



In [27]:
# Define application steps
# def retrieve(state: State):
#     retrieved_docs = vector_store.similarity_search(state["question"])
#     return {"context": retrieved_docs}


# def generate(state: State):
#     docs_content = "\n\n".join(doc.page_content for doc in state["context"])
#     messages = prompt.invoke({"question": state["question"], "context": docs_content})
#     response = llm.invoke(messages)
#     return {"answer": response} # Corrected to return the string directly


# # Compile application
# graph_builder = StateGraph(State).add_sequence([retrieve, generate])
# graph_builder.add_edge(START, "retrieve")
# graph_builder.add_edge("retrieve","generate")
# graph = graph_builder.compile()

evaluation_results = []

for index, row in qna_df.iterrows():
    question = row['question']
    expected_answer = row['answer']
    generated_answer = get_rag_answer(question)
    print(f"{index+1} Qn done")

    evaluation_results.append({
        'question': question,
        'expected_answer': expected_answer,
        'generated_answer': generated_answer
    })

print(evaluation_results)

1 Qn done
2 Qn done
3 Qn done
4 Qn done
5 Qn done
6 Qn done
7 Qn done
8 Qn done
9 Qn done
10 Qn done
[{'question': 'What is task decomposition in autonomous agents?', 'expected_answer': 'Task decomposition involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.', 'generated_answer': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know. Use three sentences maximum and keep the answer concise.\nQuestion: What is task decomposition in autonomous agents? \nContext: LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-o

## Analyze results

### Subtask:
Calculate and display metrics such as accuracy, precision, or recall based on the comparison of generated and expected answers.


**Reasoning**:
Iterate through the evaluation results, compare generated and expected answers, calculate accuracy, and print the results.



In [29]:
from sklearn.metrics.pairwise import cosine_similarity

def is_answer_correct(generated_answer: str, expected_answer: str, embedding_model) -> bool:
    """
    Compares the generated answer with the expected answer using semantic similarity.

    Args:
        generated_answer: The answer generated by the RAG model.
        expected_answer: The expected answer from the Q&A dataset.
        embedding_model: The embedding model to use for calculating semantic similarity.

    Returns:
        True if the answers are semantically similar, False otherwise.
    """
    # Convert to lowercase for case-insensitive comparison
    generated_answer_lower = generated_answer.lower()
    expected_answer_lower = expected_answer.lower()

    # Get embeddings for the generated and expected answers
    generated_embedding = embedding_model.embed_query(generated_answer_lower)
    expected_embedding = embedding_model.embed_query(expected_answer_lower)

    # Calculate cosine similarity between the embeddings
    similarity_score = cosine_similarity([generated_embedding], [expected_embedding])[0][0]

    # Define a similarity threshold (you may need to tune this value)
    similarity_threshold = 0.5

    return similarity_score >= similarity_threshold

total_questions = len(evaluation_results)
correct_answers = 0

for result in evaluation_results:
    question = result['question']
    expected_answer = result['expected_answer']
    generated_answer = result['generated_answer']

    # Pass the embedding_model to the function
    if is_answer_correct(generated_answer, expected_answer, embedding_model):
        correct_answers += 1
       # print(f"Question: {question}\nExpected: {expected_answer}\nGenerated: {generated_answer}\nResult: Correct\n")
    #else:
       # print(f"Question: {question}\nExpected: {expected_answer}\nGenerated: {generated_answer}\nResult: Incorrect\n")


accuracy = (correct_answers / total_questions) * 100 if total_questions > 0 else 0

print(f"Total questions: {total_questions}")
print(f"Correctly answered: {correct_answers}")
print(f"Accuracy: {accuracy:.2f}%")

Total questions: 10
Correctly answered: 8
Accuracy: 80.00%


## Summarize findings

### Subtask:
Present a summary of the benchmarking results and insights into the RAG model's performance.


**Reasoning**:
Summarize the benchmarking results, discuss potential reasons for the performance, and suggest areas for improvement based on the previous analysis.



In [ ]:
print("--- RAG Model Benchmarking Summary ---")
print(f"Total questions evaluated: {total_questions}")
print(f"Number of correctly answered questions: {correct_answers}")
print(f"Overall accuracy: {accuracy:.2f}%\n")

print("--- Analysis and Insights ---")
print(f"The RAG model achieved an accuracy of {accuracy:.2f}% on the provided Q&A dataset using semantic similarity.")
print("Semantic similarity provides a more nuanced evaluation than a simple substring check,")
print("accounting for variations in phrasing while capturing the core meaning.")
print("Further analysis of incorrectly answered questions can help identify areas for improvement in the RAG pipeline,")
print("such as prompt engineering, document chunking strategies, or the choice of language and embedding models.")
print("Manual evaluation of a subset of results is still recommended to gain deeper insights into the quality of generated answers.")

print("\n--- Suggested Areas for Improvement ---")
print("1. Refine the semantic similarity threshold: Experiment with different threshold values to find the best balance for your specific use case.")
print("2. Manual evaluation: Conduct a manual review of a subset of generated answers, especially those marked as incorrect by the semantic similarity metric, to understand the types of errors.")
print("3. Fine-tune the RAG model: Experiment with different prompting strategies, chunk sizes for document splitting, or different base language models and embedding models to potentially improve generation quality and semantic similarity.")
print("4. Expand the Q&A dataset: A larger and more diverse dataset would provide a more robust evaluation of the model's performance across various types of questions and phrasings.")

--- RAG Model Benchmarking Summary ---
Total questions evaluated: 1
Number of correctly answered questions: 0
Overall accuracy: 0.00%

--- Analysis and Insights ---
The RAG model achieved an accuracy of 0.00% on the provided Q&A dataset using semantic similarity.
Semantic similarity provides a more nuanced evaluation than a simple substring check,
accounting for variations in phrasing while capturing the core meaning.
Further analysis of incorrectly answered questions can help identify areas for improvement in the RAG pipeline,
such as prompt engineering, document chunking strategies, or the choice of language and embedding models.
Manual evaluation of a subset of results is still recommended to gain deeper insights into the quality of generated answers.

--- Suggested Areas for Improvement ---
1. Refine the semantic similarity threshold: Experiment with different threshold values to find the best balance for your specific use case.
2. Manual evaluation: Conduct a manual review of a su